In [2]:
import os, re
import pandas as pd
from sqlglot import parse_one, exp
from utils import get_files_absolute_path_from_dir, connect_postgres, sql_to_dataframe

In [3]:
path = os.path.join(os.getcwd(), 'all_queries', 'updated_queries')
files = os.listdir(path)
print(path)
queries = get_files_absolute_path_from_dir(path)

/home/alfio/Desktop/DataWarehouse/TPC_DS/tpcds-benchmark/all_queries/updated_queries
Total files: 99
First few files...
['/home/alfio/Desktop/DataWarehouse/TPC_DS/tpcds-benchmark/all_queries/updated_queries/query-08.sql', '/home/alfio/Desktop/DataWarehouse/TPC_DS/tpcds-benchmark/all_queries/updated_queries/query-79.sql', '/home/alfio/Desktop/DataWarehouse/TPC_DS/tpcds-benchmark/all_queries/updated_queries/query-06.sql', '/home/alfio/Desktop/DataWarehouse/TPC_DS/tpcds-benchmark/all_queries/updated_queries/query-81.sql', '/home/alfio/Desktop/DataWarehouse/TPC_DS/tpcds-benchmark/all_queries/updated_queries/query-30.sql']


In [177]:
conditions = []
def extract_conditions(expression):
    if isinstance(expression, sqlglot.exp.And) or isinstance(expression, sqlglot.exp.Or):
        extract_conditions(expression.args['this'])
        extract_conditions(expression.args['expression'])
    elif isinstance(expression, sqlglot.exp.Exists) or isinstance(expression, sqlglot.exp.Subquery):
        return
    elif isinstance(expression, sqlglot.exp.Paren):
            extract_conditions(expression.this)
    else:
        conditions.append(expression.sql())

def remove_subquery(condition):
    idx = condition.find('SELECT')
    if idx != -1:
        return condition[:idx]
    return condition


def remove_case_when(condition):
    if "CASE" in condition:
        return 0
    return 1


for query in queries:
    with open(query, "r") as f:
        all_lines = []
        for line in f:
            cleaned_line = line.strip()
            if cleaned_line != '':
                all_lines.append(cleaned_line)
                
        one_line_query = ' '.join(all_lines)
        parsed = sqlglot.parse_one(one_line_query, dialect="postgres")
        wheres = parsed.find_all(exp.Where)
        for where in wheres:
            extract_conditions(where.this)
    


conditions = list(map(remove_subquery, filter(remove_case_when ,conditions)))
    
print(len(conditions))

1327


In [158]:
cur = connect_postgres("tpcds")

PostgreSQL server information
{'user': 'postgres', 'channel_binding': 'prefer', 'dbname': 'tpcds', 'host': 'localhost', 'port': '25433', 'options': '', 'sslmode': 'prefer', 'sslcompression': '0', 'sslcertmode': 'allow', 'sslsni': '1', 'ssl_min_protocol_version': 'TLSv1.2', 'gssencmode': 'prefer', 'krbsrvname': 'postgres', 'gssdelegation': '0', 'target_session_attrs': 'any', 'load_balance_hosts': 'disable'} 

You are connected to -  ('PostgreSQL 17.0 (Debian 17.0-1.pgdg120+1) on x86_64-pc-linux-gnu, compiled by gcc (Debian 12.2.0-14) 12.2.0, 64-bit',) 



In [209]:
df = sql_to_dataframe(cur, "SELECT attname from PG_STATS WHERE schemaname = 'public';", ["column"])

In [210]:
no_operations = {}

for column in list(df['column']):
    no_operations[column] = {'e' : 0, 'c' : 0}
    for condition in conditions:
        if column in condition:
            if re.match(r".*(<|>|\bbetween\b).*", condition, flags=re.IGNORECASE):
                no_operations[column]['c'] += 1
            elif re.match(r".*(=|\bIN\b|\bIS\b|\bLIKE\b).*", condition, flags=re.IGNORECASE):
                no_operations[column]['e'] += 1
            

print(no_operations)

{'ca_address_sk': {'e': 46, 'c': 0}, 'ca_address_id': {'e': 0, 'c': 0}, 'ca_street_number': {'e': 0, 'c': 0}, 't_second': {'e': 0, 'c': 0}, 't_am_pm': {'e': 0, 'c': 0}, 'ca_street_name': {'e': 0, 'c': 0}, 'ca_street_type': {'e': 0, 'c': 0}, 'ca_suite_number': {'e': 0, 'c': 0}, 'ca_city': {'e': 1, 'c': 2}, 'ca_county': {'e': 7, 'c': 0}, 'ca_state': {'e': 18, 'c': 0}, 't_shift': {'e': 0, 'c': 0}, 't_sub_shift': {'e': 0, 'c': 0}, 'wp_autogen_flag': {'e': 0, 'c': 0}, 'ca_zip': {'e': 5, 'c': 1}, 'ca_country': {'e': 9, 'c': 1}, 'ca_gmt_offset': {'e': 12, 'c': 0}, 'ca_location_type': {'e': 0, 'c': 0}, 'cd_demo_sk': {'e': 21, 'c': 0}, 'cd_gender': {'e': 4, 'c': 0}, 'cd_marital_status': {'e': 18, 'c': 1}, 'cd_education_status': {'e': 18, 'c': 0}, 'cd_purchase_estimate': {'e': 0, 'c': 0}, 'cd_credit_rating': {'e': 0, 'c': 0}, 'cd_dep_count': {'e': 0, 'c': 0}, 'cd_dep_employed_count': {'e': 0, 'c': 0}, 'cd_dep_college_count': {'e': 0, 'c': 0}, 't_time_sk': {'e': 14, 'c': 0}, 't_time_id': {'e': 0,

In [211]:
analytics = pd.DataFrame([
    {'attname': key, 'equalities': value['e'], 'comparisons': value['c']}
    for key, value in no_operations.items()
])
analytics

,attname,equalities,comparisons
0,ca_address_sk,46,0
1,ca_address_id,0,0
2,ca_street_number,0,0
3,t_second,0,0
4,t_am_pm,0,0
...,...,...,...
308,ss_ext_tax,0,0
309,ss_coupon_amt,0,6
310,ss_net_paid,0,1
311,ss_net_paid_inc_tax,0,0
